In [ ]:
# PART 1

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
# set theurl and endpoint we want to hit
url = 'https://api.yelp.com/v3/businesses/search'
url2 = 'https://api.yelp.com/v3/businesses/{id}/reviews'

In [3]:
client_id = 'WI-KQqrlT9mjEbomtiunqQ'
api_key= "xbU_ORLfRas4oibkTbDBaz9fckkG894QHilPelxHak6XSebxCdbv3mPK1hf05y3Ymc9gvTRCIfS9d4Wt2oILXJbo3Kwx1_HRfoSDd0EnxTpDmm8WnIjPtAaVGRv2X3Yx"

In [4]:
# from  keys  import  client_id, api_key

In [5]:
print(api_key)

xbU_ORLfRas4oibkTbDBaz9fckkG894QHilPelxHak6XSebxCdbv3mPK1hf05y3Ymc9gvTRCIfS9d4Wt2oILXJbo3Kwx1_HRfoSDd0EnxTpDmm8WnIjPtAaVGRv2X3Yx


In [6]:
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

In [23]:
term = 'restaurant'
location = 'Los Angeles'
categories = 'cafe'
radius = 3500

In [24]:
url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "categories" : categories,
                "radius": radius,
                "limit": 50,
#                 "offset":offset
            }

In [25]:
url_params

{'term': 'restaurant',
 'location': 'Los+Angeles',
 'categories': 'cafe',
 'radius': 3500,
 'limit': 50}

In [26]:
response = requests.get(url, headers=headers, params=url_params)

In [27]:
response.status_code

200

In [28]:
data = response.json()
data['total']

1400

In [29]:
# PART 2

In [30]:
import sqlite3
import pandas as pd

In [15]:
conn = sqlite3.connect('yelp.db')
cursor = conn.cursor()

In [16]:
cursor.execute("""Create TABLE businesses(
                                businessid TEXT,
                                rating NUMERIC,
                                price TEXT,
                                name TEXT,
                                review_count INTEGER,
                                address TEXT,
                                is_closed TEXT
                                )
                """)
conn.commit()

In [17]:
cursor.execute("""Create TABLE reviews(
                                businessid TEXT,
                                reviewID INTEGER,
                                rating NUMERIC,
                                time_created TEXT,
                                text TEXT
                                )
                """)
conn.commit()

In [31]:
def yelp_call(url, url_params, api_key):
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()
    return data

In [32]:
results = yelp_call(url, url_params, api_key)

In [33]:
def parse_results(results):
    
    parsed_results = []
    count = 0
    
    for business in results['businesses']:
        for item in ['businessid', 'rating', 'price', 'name', 'review_count', 'address', 'is_closed']:
            if item not in business:
                business[item] = np.nan
            else:
                business[item]
    
        biz_tuple = (business['id'],
                business['rating'],
                business['price'],
                business['name'],
                business['review_count'],
                business['location']['display_address'][0],
                business['is_closed'])
        count += 1
        parsed_results.append(biz_tuple)
    
    return parsed_results

In [34]:
def db_insert(conn, cursor, parse_results):
# for business in parse_results:
    add_business = ("""
    INSERT INTO businesses (businessid, rating, price, name, review_count, address, is_closed) 
    VALUES(?, ?, ?, ?, ?, ?, ?)
    """)
    cursor.executemany(add_business, parse_results)
    conn.commit()

In [35]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url, url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(conn, cursor, parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50
    
print(cur)

1000


In [36]:
query = ("SELECT businessid FROM businesses")
cursor.execute(query)
business_ID_list = cursor.fetchall()
business_ID_list

[('IrfmgL_zZk3T_G8OzWoqMw',),
 ('_1pt3zjz2Om9QC3OUSuTtA',),
 ('eYXwVR4mMAjzkJnm5wneHQ',),
 ('TIvMX0RWgx15o5zwfWe2Pw',),
 ('kFgjHGp-cTUK5uPaSJOORA',),
 ('8kck3-K4zYKTJbJko0JlXQ',),
 ('vmZ-bgnM5B76NUpegAjy1g',),
 ('4rXyiTBn0b9xiuKkHoJ32g',),
 ('wdqRLypbCo7KCyhLNB8mIQ',),
 ('BjlVE7tTp1UCEttgkM1Big',),
 ('z4g4bZPR-Z8bFlwv9APl2w',),
 ('KA-P3d2NcbJy5SQN-vCzPA',),
 ('0tiSST3GCwTa7eqY6yTlHQ',),
 ('v6iAL-bM8rjkL1E7jIItXw',),
 ('wxzx3duu1c0CcomzEOvfOQ',),
 ('eI0DpD2Xx8hFlk7eAdFCqQ',),
 ('HlDkf_7TxPLaPn882N1Nxw',),
 ('PgJV1wOtwTuoUz_n5Mcyxw',),
 ('_hOVIgjVRl_HzvLaa65KJg',),
 ('d5752S_RDb26HxoL6O06eQ',),
 ('Vqq-m8VvLWH2pV5BZZeRoA',),
 ('PFk6k_hEGG0n6_FAJSm9EQ',),
 ('HHtpR0RslupSQ99GIIwW5A',),
 ('cCVR9W_L2rqexfKAjfBTzQ',),
 ('VcJzpsjaPi8HZ8zXI6GMOQ',),
 ('SGRmnarrNuVEsAjYdEoA0w',),
 ('v3qnV929O7VFRVUB5hd72A',),
 ('XQLmEdXoMzOpffwoFaBtaQ',),
 ('HECB3khkUffaD2zWCngLuw',),
 ('agq8VUqVQ_Y3dZh6-IjeIg',),
 ('tr15WgMHIC_W2gqBpuk3-A',),
 ('IlkvuSkI1LF5y-Q3jpWrWQ',),
 ('UNmJz8U03hjA5I5DB87BBw',),
 ('u39mZEY

In [37]:
type(business_ID_list[0][0])

str

In [38]:
# pd.read_sql("SELECT businessid FROM businesses", conn)

In [39]:
# pd.DataFrame(business_ID_list)

In [40]:
biz_id_list = []
for biz in business_ID_list:
    biz_id = biz[0]
    biz_id_list.append(biz_id)

In [41]:
biz_id_list

['IrfmgL_zZk3T_G8OzWoqMw',
 '_1pt3zjz2Om9QC3OUSuTtA',
 'eYXwVR4mMAjzkJnm5wneHQ',
 'TIvMX0RWgx15o5zwfWe2Pw',
 'kFgjHGp-cTUK5uPaSJOORA',
 '8kck3-K4zYKTJbJko0JlXQ',
 'vmZ-bgnM5B76NUpegAjy1g',
 '4rXyiTBn0b9xiuKkHoJ32g',
 'wdqRLypbCo7KCyhLNB8mIQ',
 'BjlVE7tTp1UCEttgkM1Big',
 'z4g4bZPR-Z8bFlwv9APl2w',
 'KA-P3d2NcbJy5SQN-vCzPA',
 '0tiSST3GCwTa7eqY6yTlHQ',
 'v6iAL-bM8rjkL1E7jIItXw',
 'wxzx3duu1c0CcomzEOvfOQ',
 'eI0DpD2Xx8hFlk7eAdFCqQ',
 'HlDkf_7TxPLaPn882N1Nxw',
 'PgJV1wOtwTuoUz_n5Mcyxw',
 '_hOVIgjVRl_HzvLaa65KJg',
 'd5752S_RDb26HxoL6O06eQ',
 'Vqq-m8VvLWH2pV5BZZeRoA',
 'PFk6k_hEGG0n6_FAJSm9EQ',
 'HHtpR0RslupSQ99GIIwW5A',
 'cCVR9W_L2rqexfKAjfBTzQ',
 'VcJzpsjaPi8HZ8zXI6GMOQ',
 'SGRmnarrNuVEsAjYdEoA0w',
 'v3qnV929O7VFRVUB5hd72A',
 'XQLmEdXoMzOpffwoFaBtaQ',
 'HECB3khkUffaD2zWCngLuw',
 'agq8VUqVQ_Y3dZh6-IjeIg',
 'tr15WgMHIC_W2gqBpuk3-A',
 'IlkvuSkI1LF5y-Q3jpWrWQ',
 'UNmJz8U03hjA5I5DB87BBw',
 'u39mZEYojBiNic3lqKhPNw',
 '2hGIxgprREdBieylltWaRQ',
 '4eaK2-Q0dEXWMr5SOoV_Sg',
 'kOYj2sGMZnyC1lU-_Uw5fA',
 

In [53]:
def yelp_call_reviews(ID):
    url = 'https://api.yelp.com/v3/businesses/{}/reviews'.format(ID)
    response = requests.get(url, headers=headers)
    data = response.json()
    if 'reviews' in data:
        return data['reviews']
    else:
        return None

In [43]:
yelp_call_reviews('GbAZbcCR969hfa4kCQqRcw')

[{'id': 'DjaiVl_LKXsTHHrZqxNZ7g',
  'url': 'https://www.yelp.com/biz/eat-this-cafe-los-angeles?adjust_creative=WI-KQqrlT9mjEbomtiunqQ&hrid=DjaiVl_LKXsTHHrZqxNZ7g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=WI-KQqrlT9mjEbomtiunqQ',
  'text': 'Bottomless mimosas?! Sign me uppp!!\n\nI got a chance to try Eat This Cafe pre-COVID and it was a great experience! The line does get SUPER long though since...',
  'rating': 5,
  'time_created': '2020-12-13 21:49:12',
  'user': {'id': '22GVpNvRujO8KRBGpEcgqg',
   'profile_url': 'https://www.yelp.com/user_details?userid=22GVpNvRujO8KRBGpEcgqg',
   'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/KrdyzO-IyTACWrJFoHtEJg/o.jpg',
   'name': 'Diana L.'}},
 {'id': '2bC581maEO0vq-_RadtU_g',
  'url': 'https://www.yelp.com/biz/eat-this-cafe-los-angeles?adjust_creative=WI-KQqrlT9mjEbomtiunqQ&hrid=2bC581maEO0vq-_RadtU_g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=WI-KQqrlT9mjEbomtiunqQ',
  'text': 'ALL

In [44]:
biz = biz_id_list[0]

In [ ]:
# biz

In [45]:
api_results = yelp_call_reviews(biz)

In [ ]:
# api_results

In [46]:
tuple_list = []
for x in api_results:  
    review_tuple = (
                    x['id'],
                    x['text'], 
                    x['rating'], 
                    x['time_created'],
                   )
    tuple_list.append(review_tuple)

# tuple_list

In [47]:
def parse_review(businessID, review_results):
    tuple_lists = [] 
    
    if 'error' in review_results:
            if review_results['error']['code'] == 'BUSINESS_UNAVAILABLE':
                return None
    else:
            for d in review_results:
                    review_tuple = ( businessID,
                                    d['id'], 
                                    d['rating'],
                                    d['time_created'],
                                    d['text'],
                      )
                    tuple_lists.append(review_tuple)
        
    return tuple_lists

In [ ]:
# def parse_review_results(results):
#     parsed_results = []
#     count = 0  
#     for business in results['reviews']:
#         for item in ['businessid', 'reviewID', 'rating', 'time', 'text']:
#             if item not in business:
#                 business[item] = np.nan
#             else:
#                 business[item]   
#         biz_tuple = (review['id'],
#                 review['reviewID'],
#                 review['text'],
#                 business['name'],
#                 business['review_count'],
#                 business['location']['display_address'][0],
#                 business['is_closed'])
#         count += 1
#         parsed_results.append(biz_tuple)  
#     return parsed_results

In [48]:
parse_review(biz, api_results)

[('IrfmgL_zZk3T_G8OzWoqMw',
  '0rnSrRaxmfd26QnGSn36mQ',
  5,
  '2020-12-16 03:07:41',
  'Life is definitely\u200b different during the pandemic lockdown\u200b now.  People are moving towards paradigm shift investments like streaming services, mirror...'),
 ('IrfmgL_zZk3T_G8OzWoqMw',
  'jsW1xaIXAYjG00TYhXSfSQ',
  5,
  '2020-12-12 05:10:32',
  "If you've ever enjoyed Puerto Rican food and sandwiches and have searched in vain to satisfy your cravings in San Francisco, then I have some good news,..."),
 ('IrfmgL_zZk3T_G8OzWoqMw',
  'XZSAvxkca4j9VIson9N-yQ',
  5,
  '2020-12-13 18:25:20',
  "Tripleta is DELICIOUS, y'all! I am so excited to have some Puerto Rican food in the neighborhood. \n\nLet's see... I ordered just about everything. The...")]

In [49]:
def review_insert(conn, cursor, parse_reviews):
    add_reviews = ("""
    INSERT INTO reviews(businessid, reviewID, rating, time_created, text) 
    VALUES(?, ?, ?, ?, ?)
    """)
    cursor.executemany(add_reviews, parse_reviews)
    conn.commit()

In [55]:
biz_id_list.index('Boa2T4-JplIX7ATmrnzI5w')

1746

In [59]:
biz_id_list[1746]

'Boa2T4-JplIX7ATmrnzI5w'

In [ ]:
"""SELECT reviewID, count(*) as C
FROM reviews 
group by reviewID 
HAVING C > 1"""

In [ ]:
query unique biz ids in reveiw table

write a list comprehension to give you biz 

In [61]:
for biz in biz_id_list[1746:]:
    print(biz)
    review_results = yelp_call_reviews(biz)
    if review_results == None:
        pass
    parsed_reviews = parse_review(biz, review_results)
    review_insert(conn, cursor, parsed_reviews)
    

Boa2T4-JplIX7ATmrnzI5w
cfmt0lJ0Jkt9cBq3NoRRNw
rt4nXUTE-1YEnBIioS9rsA
6keXTWRER7WbgP80Uf1IZw
4zqH3coW8sJ0OZ243fg_3A
1vsJ_u3YWCVZh0sNbO6zxw
GEs_k-cIOtcQlIqrnShStA
18zgUgvXXL5AJ74DxCG0OQ
6xwX5NVwihXnRxN9g9iqOg
mf1xl8lvsXNAsVSmCq6tvQ
5Ve-c454-qjfayf-lW4HGA
6qboq8qiSdltjiwom2xHtA
yS2N0fgGf_a7qYR3ZaZABQ
E1PKbUE_rsb6OQs0dMRh_w
jvAdkZ6ms8xkwFQzm7WV1Q
JJDXCLdhdt-SIjzKrnhJNw
svZZxiVPaOLLjjt0aPdggQ
SLMgb3AXDI5Acv7tKJA3mQ
26FRGIN7EWIBB19zWeejTQ
ijIG_jEynDV2Eg6ThuoZMQ
qkEKJgBhcBTCu5a3lgQ0-w
w0gssMlA04EBt989b0m8vg
G18cnw5WFpQWosWHKrGBew
pm5GtrZ7FbRSp0tO-Hdkog
FKQl42z8D3FoPrlTsjTFJg
boiwVvlw8u9nZ3szhiP2Cg
o4yf7OSSyOf-cpkKuS0u2w
9voac6rPhHB5z7Z2mXd89w
sNANhRsAIEq5na7i8EgsRA
3XPZ8FhWuCFY8KJANpZ8jg
eO-iERZ5u_N06pMAlVpQCg
DbGV0OH7tVUDHnPi4ZXUKg
aA0C5KJSFHltvuMfYcT3Aw
1t89y1JeBGBC4aEbBNZjRg
zemavtNcCOaA9TGf1Vmt4g
1x7pBWoIesBql1a_VzYHNw
nh-_ZaO9SU3YlYAyawdEJw
UZ0fPngKt0CHVeJe4WfUqg
THwkGysQn2_9dC2MlIWshw
N3kpw-HTFn87X7r0jvcI_A
l6EkGL_6uAodYcfdvM-UKQ
mzLOMxMrX_iNRzLcAs1SUw
51Z90o-sgugvoUGMpf5l-g
5vGerGMy1yc

In [60]:
yelp_call_reviews('Boa2T4-JplIX7ATmrnzI5w')

[{'id': 'ePFjBovvjWm-TrPRzwavoA',
  'url': 'https://www.yelp.com/biz/all-in-bowl-los-angeles?adjust_creative=WI-KQqrlT9mjEbomtiunqQ&hrid=ePFjBovvjWm-TrPRzwavoA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=WI-KQqrlT9mjEbomtiunqQ',
  'text': 'I am a Russian born in China, I have been living there for 5 years in my adulthood and got addicted to fried rice...In a while I moved to the US...Fried...',
  'rating': 5,
  'time_created': '2018-11-15 12:26:57',
  'user': {'id': 'radQp30YwrEB7TjhqOudZg',
   'profile_url': 'https://www.yelp.com/user_details?userid=radQp30YwrEB7TjhqOudZg',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/photo/q4Qt3Gzs_SV4h27f32LtWg/o.jpg',
   'name': 'Danny V.'}},
 {'id': 'gx_xSaSKXZQIrBuvZmL1aw',
  'url': 'https://www.yelp.com/biz/all-in-bowl-los-angeles?adjust_creative=WI-KQqrlT9mjEbomtiunqQ&hrid=gx_xSaSKXZQIrBuvZmL1aw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=WI-KQqrlT9mjEbomtiunqQ',
  'text': 'Not my cup o

In [ ]:
# SELECT reviewID, count(*) as C
# FROM reviews 
# group by reviewID 
# HAVING C > 1;

In [ ]:
# cursor.execute("""ALTER TABLE reviews
# RENAME COLUMN reviewID TO texto;""")


In [ ]:
# cursor.execute("""ALTER TABLE reviews
# RENAME COLUMN  text TO reviewID;""")

In [ ]:
# cursor.execute("""ALTER TABLE reviews
# RENAME COLUMN texto TO text;""")


In [68]:
# Which are the 5 most reviewed businesses?
cursor.execute("""
SELECT name, review_count
FROM businesses
ORDER BY review_count DESC
LIMIT 5
""").fetchall()

[("Brenda's French Soul Food", 11054),
 ('Tartine Bakery & Cafe', 8014),
 ("Pink's Hot Dogs", 7163),
 ('Burma Superstar', 6838),
 ('Republique', 5627)]

In [69]:
# What is the highest rating recieved in your data set and how many businesses have that rating?
cursor.execute("""
SELECT name, rating
FROM businesses
WHERE rating = 5""").fetchall()

[('Schlok’s', 5),
 ('Presidio Kebab', 5),
 ('Base Camp', 5),
 ('Routier', 5),
 ('Berliner Berliner', 5),
 ('Limoncello', 5),
 ('Komaaj', 5),
 ("Chuy's Fiestas", 5),
 ('Shish Ke Baba', 5),
 ('Califorrito', 5),
 ("AshYan's Lu Ruo Fan", 5),
 ('Sowl Bowls Thai Street Food', 5),
 ("Ap's Cafe", 5),
 ("That's My Jam", 5),
 ('Marrakech Magic Theater', 5),
 ("Jordan's Kitchen", 5),
 ('Richmond Burritos', 5),
 ('Marlena Restaurant', 5),
 ('Dabao Singapore', 5),
 ('San Ho Won', 5),
 ('RAVISHSF', 5),
 ('Limonnana', 5),
 ('Falafelland', 5),
 ('Pinche Sushi', 5),
 ('Destapas', 5),
 ('Mesopotamia Kitchen', 5),
 ('Sambal-sambal', 5),
 ("Newkirk's", 5),
 ('Above Ground', 5),
 ('Anomaly', 5),
 ('Shovels Bar', 5),
 ('Gusto Pinsa Romana', 5),
 ('Intu-on', 5),
 ('Seafood Station', 5),
 ('Kusan Uyghur Cuisine', 5),
 ('Akna', 5),
 ('Sarah Kirnon and Miss Ollie’s Presents The Cook Shop - Elda', 5),
 ('Pho Nation', 5),
 ('Peruvian Picnic', 5),
 ('The Door', 5),
 ('Saucy Bandits', 5),
 ('ChiliCali', 5),
 ('Quee

In [79]:
cursor.execute("""
SELECT name, rating
FROM businesses 
WHERE rating IN 
(SELECT MAX(rating) FROM businesses)""").fetchall()

[('Schlok’s', 5),
 ('Presidio Kebab', 5),
 ('Base Camp', 5),
 ('Routier', 5),
 ('Berliner Berliner', 5),
 ('Limoncello', 5),
 ('Komaaj', 5),
 ("Chuy's Fiestas", 5),
 ('Shish Ke Baba', 5),
 ('Califorrito', 5),
 ("AshYan's Lu Ruo Fan", 5),
 ('Sowl Bowls Thai Street Food', 5),
 ("Ap's Cafe", 5),
 ("That's My Jam", 5),
 ('Marrakech Magic Theater', 5),
 ("Jordan's Kitchen", 5),
 ('Richmond Burritos', 5),
 ('Marlena Restaurant', 5),
 ('Dabao Singapore', 5),
 ('San Ho Won', 5),
 ('RAVISHSF', 5),
 ('Limonnana', 5),
 ('Falafelland', 5),
 ('Pinche Sushi', 5),
 ('Destapas', 5),
 ('Mesopotamia Kitchen', 5),
 ('Sambal-sambal', 5),
 ("Newkirk's", 5),
 ('Above Ground', 5),
 ('Anomaly', 5),
 ('Shovels Bar', 5),
 ('Gusto Pinsa Romana', 5),
 ('Intu-on', 5),
 ('Seafood Station', 5),
 ('Kusan Uyghur Cuisine', 5),
 ('Akna', 5),
 ('Sarah Kirnon and Miss Ollie’s Presents The Cook Shop - Elda', 5),
 ('Pho Nation', 5),
 ('Peruvian Picnic', 5),
 ('The Door', 5),
 ('Saucy Bandits', 5),
 ('ChiliCali', 5),
 ('Quee

In [85]:
cursor.execute("""
SELECT rating, count(rating)
FROM businesses
GROUP BY rating
ORDER BY rating DESC
LIMIT 1
""").fetchall()

[(5, 178)]

In [100]:
# What percentage of businesses have a rating greater than or 4.5?
n=cursor.execute("""SELECT round(count(rating), 2) * 100 / 
(SELECT count(businessid) FROM businesses)
FROM BUSINESSES
WHERE rating > 4.5""").fetchall()

n[0][0]

8.9

In [105]:
# What percentage of businesses have a rating less than 3?
n=cursor.execute("""SELECT round(count(rating), 2) * 100 / 
(SELECT count(businessid) FROM businesses)
FROM BUSINESSES
WHERE rating < 3""").fetchall()

n[0][0]

0.8

In [131]:
# What is the average rating of businesses that have a price label of one dollar sign? Two dollar signs? Three dollar signs?
x=cursor.execute("""SELECT
                   price, AVG(rating)
                   FROM businesses
                   WHERE price = '$' OR price = '$$' OR price = '$$$'
                   GROUP BY price
                   """).fetchall()


In [132]:
# Return the text of the reviews for the most reviewed business.

cursor.execute("""SELECT text
                FROM reviews
                WHERE businessid = (SELECT businessid 
                FROM businesses 
                ORDER BY review_count DESC
                LIMIT 1)""").fetchall()

[("Um. You can turn any grits into cheddar/cheesy grits. 100% worth it. They were so freaking good. Even my friend who doesn't super like grits thought it...",),
 ('This was my favorite meal when I stayed in San Francisco - which is a difficult distinction to make since I ate So. Much. Deliciousness! But this place has...',),
 ("This is definitely one of the best restaurant with food of a good quality. Not my first time and I'm always satisfied with how it's all made. So attentive...",)]

In [118]:
# Return the name of the business with the most recent review.

cursor.execute("""SELECT name
                FROM businesses
                WHERE businessid = (SELECT businessid 
                FROM reviews 
                ORDER BY time_created DESC
                LIMIT 1)""").fetchall()

[('Tacos El Compita',)]

In [123]:
# Find the highest rated business and return text of the most recent review. If multiple business have the same rating, 
# select the business with the most reviews.
cursor.execute("""SELECT b.name, r.text, b.rating, r.time_created, b.review_count
                FROM reviews AS r
                LEFT JOIN businesses AS b 
                ON b.businessid = r.businessid
                ORDER BY b.rating DESC, b.review_count DESC, r.time_created DESC
                LIMIT 1""").fetchall()

[('Limoncello',
  'One of the best sandwiches I have had in the Bay Area ! I love the vibe of the sandwich shop too . It has a selection of liquor and beer too . Wish you...',
  5,
  '2020-12-19 17:07:48',
  1048)]

In [125]:
# Find the lowest rated business and return text of the most recent review. If multiple business have the same rating, 
# select the business with the least review

cursor.execute("""SELECT b.name, r.text, b.rating, r.time_created, b.review_count
                FROM reviews AS r
                LEFT JOIN businesses AS b 
                ON b.businessid = r.businessid
                ORDER BY b.rating, b.review_count, r.time_created DESC
                LIMIT 1""").fetchall()

[('615 Food Court',
  "DO NOT EVER ORDER FOOD FROM HERE! If you're working for a delivery company, AVOID accepting orders from this location. \n\nIt's ALWAYS busy, orders ALWAYS...",
  1.5,
  '2020-12-14 23:14:44',
  15)]